# Baseline (01)

# Imports

In [30]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#from wcs.google import google_drive_share
import pandas as pd
import urllib.request
from urllib.parse import urlparse

#from google.colab import drive


import warnings
warnings.simplefilter(action='ignore')
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [31]:
!pip install gdown

# Load Data

In [32]:
BASE_DIR ="./"
IMAGES_DIR = "/kaggle/input/movie-poster-genre-2021/images/"

# Preproc

In [33]:
def retrieveFilename(url):
    a = urlparse(url)
    picfilename = os.path.basename(a.path)
    return picfilename


In [49]:
import gdown

parquet_fname = "./df.parquet.gzip"
if not os.path.exists(parquet_fname):
    url = 'https://drive.google.com/uc?id=178mXeXUC1400lj-LrWyhD3yfW5_KsVYw'
    gdown.download(url, parquet_fname, quiet=False)
    df = pd.read_parquet("./df.parquet.gzip")
    
    df["filename"] = df.loc[~df["poster_url"].isnull(),"poster_url"].apply(retrieveFilename)
    
    
    # use only mot null rows
    df = df.dropna()
    # set data path to basename of the file
    #df['data_path'] = df['poster_path'].apply(lambda x: x.split('/')[-1])
    # remove rows with empty genre_id list and set correct list type
    df['genre_id'] = df['genre_id'].apply(lambda x: np.nan if len(eval(x)) == 0 else x)
    df = df.dropna()
    df['genre_id'] = df['genre_id'].apply(lambda x: eval(x))

    print(f'len of df: {len(df)}')
    df["file_exists"] = df["filename"].apply(lambda x: os.path.exists(IMAGES_DIR + x))
    df.to_parquet(parquet_fname,compression='gzip')
else:
    df = pd.read_parquet("./df.parquet.gzip")

Downloading...
From: https://drive.google.com/uc?id=178mXeXUC1400lj-LrWyhD3yfW5_KsVYw
To: /kaggle/working/df.parquet.gzip
21.7MB [00:00, 112MB/s]


len of df: 267974


In [48]:
#!rm $parquet_fname

In [53]:
df.head()
#print(df.shape)

,adult,id,original_title,popularity,video,url,poster_url,genre_id,filename,file_exists
380003,False,537250,¿Por qué ya no me quieres?,1.40,False,https://www.themoviedb.org/movie/537250,https://www.themoviedb.org/t/p/w500//8Y0aOxqSe...,"[80, 18]",8Y0aOxqSetK3hXKPFDTrHwu6dWa.jpg,False
380010,False,537266,Success,0.60,False,https://www.themoviedb.org/movie/537266,https://www.themoviedb.org/t/p/w500//tJMm1JJUe...,[18],tJMm1JJUeWOLb4nPDBD6sZvFYU3.jpg,False
380017,False,537274,Предел возможного,0.62,False,https://www.themoviedb.org/movie/537274,https://www.themoviedb.org/t/p/w500//h9aXCUZ6g...,[18],h9aXCUZ6gj6V5NFv8a31J1OiDjq.jpg,False
380020,False,537277,母親,0.60,False,https://www.themoviedb.org/movie/537277,https://www.themoviedb.org/t/p/w500//3Ogn8SWj2...,[18],3Ogn8SWj29GpGHNSIkMYiKZdeBL.jpg,True
380022,False,537281,1+1=3 Ora Tinjon,0.60,False,https://www.themoviedb.org/movie/537281,https://www.themoviedb.org/t/p/w500//47yYJPL2q...,[18],47yYJPL2qU9LNQ2rTMs7p3RtfR7.jpg,True


In [54]:
#keep only rows where file exists in data set
df = df.loc[df["file_exists"]]

 Create ImageGenerators

In [55]:
IMAGES_DIR

'/kaggle/input/movie-poster-genre-2021/images/'

In [56]:
datagen = ImageDataGenerator(rescale=1 / 255., validation_split=0.1)
BATCH_SIZE = 64
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=IMAGES_DIR,
    x_col="filename",
    y_col="genre_id",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="categorical",    
    target_size=(299, 299),
    subset='training',
    validate_filenames=True
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=IMAGES_DIR,
    x_col="filename",
    y_col="genre_id",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(299, 299),
    subset='validation',
    validate_filenames=True
)

Found 47099 validated image filenames belonging to 19 classes.
Found 5233 validated image filenames belonging to 19 classes.


In [57]:
# show class indicies
print(train_generator.class_indices)
print('length:', len(train_generator.class_indices))


{12: 0, 14: 1, 16: 2, 18: 3, 27: 4, 28: 5, 35: 6, 36: 7, 37: 8, 53: 9, 80: 10, 99: 11, 878: 12, 9648: 13, 10402: 14, 10749: 15, 10751: 16, 10752: 17, 10770: 18}
length: 19


In [58]:
list(train_generator.class_indices.keys())

[12,
 14,
 16,
 18,
 27,
 28,
 35,
 36,
 37,
 53,
 80,
 99,
 878,
 9648,
 10402,
 10749,
 10751,
 10752,
 10770]

In [59]:
#https://datascience.stackexchange.com/questions/13490/how-to-set-class-weights-for-imbalanced-classes-in-keras
from sklearn.utils import class_weight
#In order to calculate the class weight do the following

class_weights = class_weight.compute_class_weight('balanced',
                                                  np.array(list(train_generator.class_indices.keys()),dtype="int"), 
                                                  np.array(df.genre_id.explode(),dtype="int"))
                                                 #np.unique(y_train),
                                                 #y_train)
class_weights_genre_id = dict(zip(list(train_generator.class_indices), class_weights))
display(class_weights_genre_id)
class_weights = dict(zip(list(range(len(class_weights))), class_weights))
class_weights

{12: 1.8113321781248042,
 14: 2.3895177504200062,
 16: 1.2341536273115221,
 18: 0.2402972383861828,
 27: 1.1115794598472812,
 28: 0.991826329507522,
 35: 0.3638830521199005,
 36: 3.3799914293505786,
 37: 5.303563787517574,
 53: 1.00403879090867,
 80: 1.4216589106639574,
 99: 0.45824068450101674,
 878: 2.2318516231928798,
 9648: 2.588644229621673,
 10402: 1.1714644487051389,
 10749: 0.8992454551107979,
 10751: 1.6204288222330134,
 10752: 4.459344161184211,
 10770: 2.53687134502924}

{0: 1.8113321781248042,
 1: 2.3895177504200062,
 2: 1.2341536273115221,
 3: 0.2402972383861828,
 4: 1.1115794598472812,
 5: 0.991826329507522,
 6: 0.3638830521199005,
 7: 3.3799914293505786,
 8: 5.303563787517574,
 9: 1.00403879090867,
 10: 1.4216589106639574,
 11: 0.45824068450101674,
 12: 2.2318516231928798,
 13: 2.588644229621673,
 14: 1.1714644487051389,
 15: 0.8992454551107979,
 16: 1.6204288222330134,
 17: 4.459344161184211,
 18: 2.53687134502924}

In [60]:
map_gender={"28":"Action",
"12":"Adventure",
"16":"Animation",
"35":"Comedy",
"80":"Crime",
"99":"Documentary",
"18":"Drama",
"10751":"Family",
"14":"Fantasy",
"36":	"History",
"27":"Horror",
"10402"	:"Music",
"9648":"Mystery",
"10749":"Romance",
"878"	:"Science Fiction",
"10770":"TV Movie",
"53":"Thriller",
"10752":"War",
"37":"Western"}

series_genre_id_counts = df.genre_id.explode().value_counts()
series_genre_id_counts
df_genre = pd.DataFrame(series_genre_id_counts)
df_genre["id"] = df_genre.index
df_genre.rename(columns={"genre_id" : "count"},inplace=True)
df_genre["name"] = df_genre["id"].apply(lambda x : map_gender[str(x)])
df_genre["weight"] = df_genre["id"].apply(lambda x : class_weights_genre_id[x])
df_genre.sort_values(by="count")

,count,id,name,weight
37,861,37,Western,5.303564
10752,1024,10752,War,4.459344
36,1351,36,History,3.379991
9648,1764,9648,Mystery,2.588644
10770,1800,10770,TV Movie,2.536871
14,1911,14,Fantasy,2.389518
878,2046,878,Science Fiction,2.231852
12,2521,12,Adventure,1.811332
10751,2818,10751,Family,1.620429
80,3212,80,Crime,1.421659


# Simple Model

model = keras.Sequential(
    [
      layers.Conv2D(32, (3, 3), padding='same', input_shape=(299, 299, 3)),
      layers.Activation('relu'),
      layers.Conv2D(32, (3, 3)),
      layers.Activation('relu'),
      layers.MaxPooling2D(pool_size=(2, 2)),
      layers.Dropout(0.25),
      layers.Conv2D(64, (3, 3), padding='same'),
      layers.Activation('relu'),
      layers.Conv2D(64, (3, 3)),
      layers.Activation('relu'),
      layers.MaxPooling2D(pool_size=(2, 2)),
      layers.Dropout(0.25),
      layers.Flatten(),
      layers.Dense(512),
      layers.Activation('relu'),
      layers.Dropout(0.5),
      layers.Dense(len(train_generator.class_indices), activation='sigmoid')
    ]
)


#model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])

model = keras.Sequential(
    [ 
        layers.Conv2D(32, (3, 3), padding='same', input_shape=(299, 299, 3)),
        layers.Activation('relu'),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.25),
        layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(len(train_generator.class_indices), activation='sigmoid')
        #layers.Dense(len(train_generator.class_indices), activation='softmax')

    ])

In [126]:
#https://machinelearningmastery.com/how-to-use-transfer-learning-when-developing-convolutional-neural-network-models/
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet169

from keras.models import Model

# load model
#model = VGG16(include_top=False,  input_shape=(299,299, 3))
model = DenseNet169(include_top=False,  input_shape=(299,299, 3))

# summarize the model
model.summary()


Model: "densenet169"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_20 (ZeroPadding2 (None, 305, 305, 3)  0           input_11[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 150, 150, 64) 9408        zero_padding2d_20[0][0]          
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 150, 150, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

conv5_block9_0_bn (BatchNormali (None, 9, 9, 896)    3584        conv5_block8_concat[0][0]        
__________________________________________________________________________________________________
conv5_block9_0_relu (Activation (None, 9, 9, 896)    0           conv5_block9_0_bn[0][0]          
__________________________________________________________________________________________________
conv5_block9_1_conv (Conv2D)    (None, 9, 9, 128)    114688      conv5_block9_0_relu[0][0]        
__________________________________________________________________________________________________
conv5_block9_1_bn (BatchNormali (None, 9, 9, 128)    512         conv5_block9_1_conv[0][0]        
__________________________________________________________________________________________________
conv5_block9_1_relu (Activation (None, 9, 9, 128)    0           conv5_block9_1_bn[0][0]          
__________________________________________________________________________________________________
conv5_bloc

Finally, we implemented a standard DenseNet-169 architecture with similar modifications. The final
fully-connected layer of 1000 units was once again replaced by 3 sequential fully-connected layers of
3
1024, 128, and 7 units with ReLU, ReLU, and sigmoid activations respectively. The entire model
consists of 14,479,943 parameters, out of which, 14,321,543 were trainable.

In [127]:
x= layers.Flatten()(model.layers[-1].output)
#x= layers.Dense(128, activation='relu')(x)
x= layers.Dense(1024, activation='relu')(x)
x= layers.Dense(128, activation='relu')(x)
#x= layers.Dropout(0.5)(x)
classifications = layers.Dense(len(train_generator.class_indices), activation='sigmoid')(x)


In [128]:
model = Model(inputs=model.inputs, outputs=classifications)
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_20 (ZeroPadding2 (None, 305, 305, 3)  0           input_11[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 150, 150, 64) 9408        zero_padding2d_20[0][0]          
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 150, 150, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

__________________________________________________________________________________________________
conv2_block6_0_relu (Activation (None, 75, 75, 224)  0           conv2_block6_0_bn[0][0]          
__________________________________________________________________________________________________
conv2_block6_1_conv (Conv2D)    (None, 75, 75, 128)  28672       conv2_block6_0_relu[0][0]        
__________________________________________________________________________________________________
conv2_block6_1_bn (BatchNormali (None, 75, 75, 128)  512         conv2_block6_1_conv[0][0]        
__________________________________________________________________________________________________
conv2_block6_1_relu (Activation (None, 75, 75, 128)  0           conv2_block6_1_bn[0][0]          
__________________________________________________________________________________________________
conv2_block6_2_conv (Conv2D)    (None, 75, 75, 32)   36864       conv2_block6_1_relu[0][0]        
__________

In [129]:
# mark loaded layers as not trainable
# except last layer

leng = len(model.layers)
print(leng)
for i,layer in enumerate(model.layers):
    if leng-i == 500:
      print("stopping at",i)
      break
    layer.trainable = False

    
model.summary()

599
stopping at 299
Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_20 (ZeroPadding2 (None, 305, 305, 3)  0           input_11[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 150, 150, 64) 9408        zero_padding2d_20[0][0]          
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 150, 150, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________

Total params: 150,796,371
Trainable params: 146,573,523
Non-trainable params: 4,222,848
__________________________________________________________________________________________________


In [130]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [131]:
from keras import metrics
#https://neptune.ai/blog/keras-metrics
model.compile(optimizer='adam', loss="binary_crossentropy", 
              metrics=[metrics.categorical_accuracy,
                       tf.keras.metrics.AUC(),
                        tf.keras.metrics.Precision(), 
                        tf.keras.metrics.Recall(),
                        f1_m,
                      ])

In [132]:
import os.path

fname_model = BASE_DIR + "DenseNet169_1"

if  os.path.exists(fname_model) :
    print("Load model")
    model = keras.models.load_model(fname_model)


# Train

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=20,# 5,
    #batch_size=BATCH_SIZE,
    #steps_per_epoch=1430,
    class_weight = class_weights
)

Epoch 1/20
 42/736 [>.............................] - ETA: 8:27 - loss: 3.1441 - categorical_accuracy: 0.1146 - auc_9: 0.5514 - precision_9: 0.1319 - recall_9: 0.1794 - f1_m: 0.1318

In [ ]:
model.save(fname_model)

In [ ]:
#num_samples // batch_size
183069 / 32 / 4



Epoch 1/50
520/520 [==============================] - 174s 334ms/step - loss: 0.7014 - accuracy: 0.2550 - val_loss: 0.5431 - val_accuracy: 0.3258
Epoch 2/50
520/520 [==============================] - 172s 331ms/step - loss: 0.5158 - accuracy: 0.2788 - val_loss: 0.4592 - val_accuracy: 0.3258
Epoch 3/50
520/520 [==============================] - 172s 331ms/step - loss: 0.4422 - accuracy: 0.2797 - val_loss: 0.4133 - val_accuracy: 0.3258
Epoch 4/50
520/520 [==============================] - 171s 330ms/step - loss: 0.4013 - accuracy: 0.2786 - val_loss: 0.3887 - val_accuracy: 0.3258